<a href="https://colab.research.google.com/github/projectDevloperMahima/Python/blob/main/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython-autotime
%load_ext autotime

In [ ]:
!pip install bing-image-downloader

In [ ]:
!mkdir images

In [ ]:
from bing_image_downloader import downloader
downloader.download("autorickshaws",limit=30,output_dir='images',adult_filter_off=True)

In [ ]:
from bing_image_downloader import downloader
downloader.download("rose ",limit=30,output_dir='images',adult_filter_off=True)

In [ ]:
from bing_image_downloader import downloader
downloader.download("preety sunflower ",limit=30,output_dir='images',adult_filter_off=True)

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize

datadir = '/content/images'

categories = ['autorickshaws','preety sunflower ','rose '] # Name the folder with these categories
target = []
flat_data = []

for i in categories:
  class_num= categories.index(i)
  path = os.path.join(datadir,i)
  #print(path)
  for img in os.listdir(path):
    #print(img)
    img_array = imread(os.path.join(path,img))

    img_resize = resize(img_array,(150,150,3)) # Value gets converted 0-1 from 0-255
    flat_data.append(np.ndarray.flatten(img_resize))
    
    target.append(class_num)
    
    

flat_data = np.array(flat_data)
target = np.array(target)
#images = np.array(images)

In [ ]:
len(flat_data[0])

In [ ]:
target

In [ ]:
unique,count = np.unique(target,return_counts=True)
plt.bar(categories,count)

In [ ]:
#Split data into Training and testing
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(flat_data,target,
                                                  test_size=0.3,random_state=109)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
param_grid = [
              {'C':[1,10,100,1000],'kernel':['linear']},
              {'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']},
            ]

svc = svm.SVC(probability=True)
clf = GridSearchCV(svc,param_grid)
clf.fit(x_train,y_train)


In [ ]:
y_pred = clf.predict(x_test)
y_pred

In [ ]:
y_test

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
accuracy_score(y_pred,y_test)

In [ ]:
confusion_matrix(y_pred,y_test)

In [ ]:
import pickle
pickle.dump(clf,open('img_model.p','wb'))

In [ ]:
model = pickle.load(open('img_model.p','rb'))

In [ ]:
flat_data = []
url = input('Enter your url')
img = imread(url)
img_resize = resize(img_array,(150,150,3)) # Value gets converted 0-1 from 0-255
flat_data.append(np.ndarray.flatten(img_resize))
flat_data = np.array(flat_data)
print(img.shape)
plt.imshow(img_resize)   
y_out = model.predict(flat_data)
y_out = categories[y_out[0]]
print(f'PREDICTED OUTPUT :{y_out}')

In [ ]:
!pip install streamlit

!pip install pyngrok
from pyngrok import ngrok

In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import pickle
from PIL import Image
st.title('Image classifier using Machine Learning')
st.text('Upload the Image')

model = pickle.load(open('img.model.p','rb'))
uploaded_file = st.file_uploader("Choose an image...",type= "jpg")
if uploaded_file is not None:
  img = Image.open(uploaded_file)
  st.image(img,caption='Uploaded Image')
  if st.button('Predict'):
    categories = ['autorickshaws','preety sunflower ','rose ']
    st.write('Result..')
    flat_data=[]
    img = np.array(img)
    img_resize = resize(img_array,(150,150,3)) # Value gets converted 0-1 from 0-255
    flat_data.append(np.ndarray.flatten(img_resize))
    flat_data = np.array(flat_data)  
    y_out = model.predict(flat_data)
    y_out = categories[y_out[0]]
    st.title(f'PREDICTED OUTPUT :{y_out}')


In [ ]:
!nohup streamlit run app.py &
url = ngrok.connect(port='8501')
url